# Make All Tables for Main Manuscript

Kendra Wyant  
October 3, 2025

In [ ]:

suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(source("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true"))
library(kableExtra)



Attaching package: 'kableExtra'

The following object is masked from 'package:dplyr':

    group_rows

## Data and calculations

Table 1

In [ ]:
study_dates <- read_csv(here::here(path_shared, "study_dates_messages.csv"),
                        show_col_types = FALSE)

screen <- read_csv(file.path(path_shared, "screen.csv"), 
                   col_types = cols()) |>
  filter(subid %in% study_dates$subid) |> 
  mutate(across(dsm5_1:dsm5_11, ~ recode(., "No" = 0, "Yes" = 1))) |>  
  rowwise() |>  
  mutate(dsm5_total = sum(c(dsm5_1, dsm5_2, dsm5_3, dsm5_4, dsm5_5, dsm5_6, dsm5_7, 
                              dsm5_8, dsm5_9, dsm5_10, dsm5_11))) |>  
  ungroup()

# lapses <- read_csv(file.path(path_shared, "lapses.csv"),
#                    col_types = cols()) |>
#   filter(exclude == FALSE) |>
#   filter(subid %in% study_dates$subid)

# Calcs to make df for table 1 
n_total <- nrow(study_dates)

dem_age <- screen |>
  summarise(mean = as.character(round(mean(dem_1, na.rm = TRUE), 1)),
            SD = as.character(round(sd(dem_1, na.rm = TRUE), 1)),
            min = as.character(min(dem_1, na.rm = TRUE)),
            max = as.character(max(dem_1, na.rm = TRUE))) |>
  mutate(var = "Age",
         n = as.numeric(""),
         perc = as.numeric("")) |>
  select(var, n, perc, everything()) 

dem_sex <-  screen |>
  select(var = dem_2) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |> 
  add_row(var = "Sex at Birth", .before = 1)

dem_race <- screen |>
  select(var = dem_3) |>
  mutate(var = fct_relevel(factor(var,
                         c("American Indian/Alaska Native", "Asian", "Black/African American",
                           "White/Caucasian", "Other/Multiracial")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Race", .before = 1)
  
  
dem_ethnicity <- screen |>
  select(var = dem_4) |>
  mutate(var = case_when(var == "No, I am not of Hispanic, Latino, or Spanish origin" ~ "No",
                         TRUE ~ "Yes"),
         var = fct_relevel(factor(var, c("Yes", "No")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Hispanic, Latino, or Spanish origin", .before = 1)

dem_education <- screen |>
  select(var = dem_5) |>
  mutate(var = fct_relevel(factor(var,
                         c("Less than high school or GED degree", "High school or GED",
                           "Some college", "2-Year degree", "College degree", "Advanced degree")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Education", .before = 1)

dem_employment <- screen |>
  select(var = dem_6, dem_6_1) |>
  mutate(var = case_when(dem_6_1 == "Full-time" ~ "Employed full-time",
                         dem_6_1 == "Part-time" ~ "Employed part-time",
                         TRUE ~ var)) |>
  mutate(var = fct_relevel(factor(var,
                         c("Employed full-time", "Employed part-time", "Full-time student",
                           "Homemaker", "Disabled", "Retired", "Unemployed",
                           "Temporarily laid off, sick leave, or maternity leave",
                           "Other, not otherwise specified")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Employment", .before = 1)

dem_income <- screen |>
  summarise(mean = format(round(mean(dem_7, na.rm = TRUE), 0), big.mark = ","),
            SD = format(round(sd(dem_7, na.rm = TRUE), 0), big.mark = ","),
            min =format(round(min(dem_7, na.rm = TRUE), 0), big.mark = ","),
            max = format(round(max(dem_7, na.rm = TRUE), 0), scientific = FALSE, big.mark = ",")) |>
  mutate(var = "Personal Income",
        n = as.numeric(""),
        perc = as.numeric(""),
        mean = str_c("$", as.character(mean)),
        SD = str_c("$", as.character(SD)),
        min = str_c("$", as.character(min)),
        max = as.character(max)) |>
  select(var, n, perc, everything())

dem_marital <- screen |>
  select(var = dem_8) |>
  mutate(var = case_when(var == "Never Married" ~ "Never married",
                         TRUE ~ var)) |>
  mutate(var = fct_relevel(factor(var,
                         c("Never married", "Married", "Divorced", "Separated",
                           "Widowed")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |> 
  add_row(var = "Marital Status", .before = 1)

table_dem <- dem_age |> 
  bind_rows(dem_sex) |> 
  bind_rows(dem_race) |> 
  bind_rows(dem_ethnicity) |> 
  bind_rows(dem_education) |> 
  bind_rows(dem_employment) |> 
  bind_rows(dem_income) |> 
  bind_rows(dem_marital) |> 
  mutate(range = str_c(min, "-", max),
         perc = round(perc, 1)) |> 
  select(-c(min, max)) |> 
  rename(` ` = var,
         N = n,
         `%` = perc,
         M = mean, 
         Range = range)


### Table 1: Features

In [ ]:
library(knitr)

df <- data.frame(
  `Raw Predictor` = c(
    "Originated",
    "Call duration",
    "Call answered",
    "Date/time of communication",
    "Phone number",
    "Type of Relationship",
    "Have you drank alcohol with this person?",
    "What is their drinking status?",
    "Would you expect them to drink in your presence?",
    "Are they currently in recovery from drugs or alcohol?",
    "Are they supportive about your recovery goals?",
    "How are your typical experiences with this person?",
    "DSM-5 symptom count",
    "Past year alcohol problems",
    "Craving",
    "Abstinence self-efficacy: Negative affect, social, physical, and craving subscales",
    "Number of individual alcohol counseling sessions attended (past 30 days)",
    "Number of group alcohol counseling sessions attended (past 30 days)",
    "Number of self-help group meetings attended (past 30 days)",
    "Number of other mental health counseling sessions attended (past 30 days)",
    "Number of days in contact with supportive people (past 30 days)",
    "Number of days in contact with unsupportive people (past 30 days)",
    "Taken prescribed medication for alcohol use disorder (past 30 days)",
    "Taken prescribed medication for other mental health disorder (past 30 days)",
    "Satisfaction with progress toward recovery goals (past 30 days)",
    "Confidence in abstinence ability (next 30 days)",
    "Has a goal of abstinence"
  ),
  
  `Response Options` = c(
    "Incoming, outgoing",
    "Duration (in minutes)",
    "Yes, no",
    "Date and time",
    "Phone number",
    "Family, friend, counselor or social worker, co-worker",
    "Never/almost never, occasionally, almost always/always",
    "Drinker, non-drinker, don’t know",
    "Yes, no, uncertain",
    "Yes, no, don’t know",
    "Supportive, unsupportive, mixed, neutral, don’t know",
    "Pleasant, unpleasant, mixed, neutral",
    "Numeric (4-11)",
    "Numeric (0-27)",
    "Numeric (0-30)",
    "Numeric (0-20)",
    "Numeric",
    "Numeric",
    "Numeric",
    "Numeric",
    "Numeric",
    "Numeric",
    "Yes, no",
    "Yes, no",
    "Numeric (0-4)",
    "Numeric (0-4)",
    "Yes, no, uncertain"
  ),
  
  `Feature Engineering` = c(
    "Difference and raw rate counts for text messages and voice calls",
    "Difference and raw rate sums of duration, difference and raw most recent duration",
    "Difference and raw rate counts for unanswered incoming voice calls",
    "Difference and raw rate counts for text messages and voice calls at night (10 pm – 6am) and on weekends",
    "Difference and raw rate counts of unique phone numbers",
    "Difference and raw rate counts of unique phone numbers",
    "Difference and raw rate counts of each response option",
    "Difference and raw rate counts of each response option",
    "Difference and raw rate counts of each response option",
    "Difference and raw rate counts of each response option",
    "Difference and raw rate counts of each response option",
    "Difference and raw rate counts of each response option",
    "", "", "", "",
    "", "", "", "",
    "", "",
    "Dummy coded",
    "Dummy coded",
    "",
    "",
    "Dummy coded"
  ),
  
  `Scoring Epochs` = c(
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "24, 48, 72, and 168 hours (night), 168 hours (weekend)",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "6, 12, 24, 48, 72, and 168 hours",
    "", "", "", "",
    "", "", "", "",
    "", "",
    "", "",
    "", "",
    ""
  ),
  
  `Total Features` = c(
    48, 14, 12, 20, 12, 48, 36, 36, 36, 36, 60, 48,
    1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2
  ),
  
  `Full Model` = rep("Yes", 27),
  
  `Baseline Model` = c(
    rep("No", 12),
    rep("Yes", 15)
  )
)


In [ ]:

df |> 
  knitr::kable() |> 
  kable_classic() 


Raw.Predictor,Response.Options,Feature.Engineering,Scoring.Epochs,Total.Features,Full.Model,Baseline.Model
Originated,"Incoming, outgoing",Difference and raw rate counts for text messages and voice calls,"6, 12, 24, 48, 72, and 168 hours",48,Yes,No
Call duration,Duration (in minutes),"Difference and raw rate sums of duration, difference and raw most recent duration","6, 12, 24, 48, 72, and 168 hours",14,Yes,No
Call answered,"Yes, no",Difference and raw rate counts for unanswered incoming voice calls,"6, 12, 24, 48, 72, and 168 hours",12,Yes,No
Date/time of communication,Date and time,Difference and raw rate counts for text messages and voice calls at night (10 pm – 6am) and on weekends,"24, 48, 72, and 168 hours (night), 168 hours (weekend)",20,Yes,No
Phone number,Phone number,Difference and raw rate counts of unique phone numbers,"6, 12, 24, 48, 72, and 168 hours",12,Yes,No
Type of Relationship,"Family, friend, counselor or social worker, co-worker",Difference and raw rate counts of unique phone numbers,"6, 12, 24, 48, 72, and 168 hours",48,Yes,No
Have you drank alcohol with this person?,"Never/almost never, occasionally, almost always/always",Difference and raw rate counts of each response option,"6, 12, 24, 48, 72, and 168 hours",36,Yes,No
What is their drinking status?,"Drinker, non-drinker, don’t know",Difference and raw rate counts of each response option,"6, 12, 24, 48, 72, and 168 hours",36,Yes,No
Would you expect them to drink in your presence?,"Yes, no, uncertain",Difference and raw rate counts of each response option,"6, 12, 24, 48, 72, and 168 hours",36,Yes,No
Are they currently in recovery from drugs or alcohol?,"Yes, no, don’t know",Difference and raw rate counts of each response option,"6, 12, 24, 48, 72, and 168 hours",36,Yes,No


### Table 2: Demographics

In [ ]:

footnote_table_dem_a <- "N = 144"


table_dem |> 
  knitr::kable() |> 
  kable_classic() |> 
  kableExtra::group_rows(start_row = 3, end_row = 4) |> 
  kableExtra::group_rows(start_row = 6, end_row = 10) |> 
  kableExtra::group_rows(start_row = 12, end_row = 13) |> 
  kableExtra::group_rows(start_row = 15, end_row = 20) |> 
  kableExtra::group_rows(start_row = 22, end_row = 30) |> 
  kableExtra::group_rows(start_row = 33, end_row = 37) |> 
  kableExtra::footnote(general = c(footnote_table_dem_a), escape=FALSE)


N 
 % 
 M 
 SD 
 Range 
 
 
 
 
 Age 
 
 
 40.4 
 11.8 
 21-72 
 
 
 Sex at Birth 
 
 
 
 
 
 
 
 
 Female 
 74 
 51.4 
 
 
 
 
 
 Male 
 70 
 48.6 
 
 
 
 
 
 Race 
 
 
 
 
 
 
 
 
 American Indian/Alaska Native 
 3 
 2.1 
 
 
 
 
 
 Asian 
 2 
 1.4 
 
 
 
 
 
 Black/African American 
 8 
 5.6 
 
 
 
 
 
 White/Caucasian 
 125 
 86.8 
 
 
 
 
 
 Other/Multiracial 
 6 
 4.2 
 
 
 
 
 
 Hispanic, Latino, or Spanish origin 
 
 
 
 
 
 
 
 
 Yes 
 3 
 2.1 
 
 
 
 
 
 No 
 141 
 97.9 
 
 
 
 
 
 Education 
 
 
 
 
 
 
 
 
 Less than high school or GED degree 
 1 
 0.7 
 
 
 
 
 
 High school or GED 
 14 
 9.7 
 
 
 
 
 
 Some college 
 39 
 27.1 
 
 
 
 
 
 2-Year degree 
 13 
 9.0 
 
 
 
 
 
 College degree 
 55 
 38.2 
 
 
 
 
 
 Advanced degree 
 22 
 15.3 
 
 
 
 
 
 Employment 
 
 
 
 
 
 
 
 
 Employed full-time 
 70 
 48.6 
 
 
 
 
 
 Employed part-time 
 25 
 17.4 
 
 
 
 
 
 Full-time student 
 7 
 4.9 
 
 
 
 
 
 Homemaker 
 1 
 0.7 
 
 
 
 
 
 Disabled 
 7 
 4.9 
 
 
 
 
 
 Retired 
 8 
 5.6 
 
 
 
 
 
 Unemployed 
 15 
 10.4 
 
 
 
 
 
 Temporarily laid off, sick leave, or maternity leave 
 3 
 2.1 
 
 
 
 
 
 Other, not otherwise specified 
 8 
 5.6 
 
 
 
 
 
 Personal Income 
 
 
 $35,050 
 $32,069 
 $0-200,000 
 
 
 Marital Status 
 
 
 
 
 
 
 
 
 Never married 
 63 
 43.8 
 
 
 
 
 
 Married 
 32 
 22.2 
 
 
 
 
 
 Divorced 
 42 
 29.2 
 
 
 
 
 
 Separated 
 5 
 3.5 
 
 
 
 
 
 Widowed 
 2 
 1.4 
 
 
 
 
 
 
 Note: 
 
 N = 144